In [5]:
import numpy as np
import pandas as pd
from random import random
import os
import librosa
import soundfile as sf
import IPython
from scipy.linalg import svd
from numpy import load, zeros, ones, asarray, vstack
from numpy.random import randint
import tensorflow as tf
import keras
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model, load_model
import keras.layers as layers
from keras.layers import Conv2D, Conv2DTranspose,BatchNormalization, LeakyReLU, Activation, Concatenate, Layer, InputSpec, Input, Conv1D, Conv1DTranspose
from keras import initializers, regularizers, constraints, backend as K
from matplotlib import pyplot
from music21 import midi
from midiutil import MIDIFile
import fluidsynth
from pydub import AudioSegment
import IPython.display as ipd
from pretty_midi import PrettyMIDI

In [2]:
generator = load_model('trained/gatob_300_model.h5')

In [8]:
test_np = np.load(r'dataset/JC_C/test/classic_piano_test_777.npy')
test_np = test_np.reshape(1,64,84,1)

B_hat = generator.predict(test_np)
B_hat = B_hat.reshape(64,84,1)

def piano_roll_to_midi(piano_roll, tempo=120, output_file='output.mid'):
    num_notes, num_steps = len(piano_roll[0]), len(piano_roll)
    midi = MIDIFile(1)
    midi.addTempo(0, 0, tempo)
    midi.addProgramChange(0, 0, 0, 0)
    for time in range(num_steps):
        for pitch in range(num_notes):
            if piano_roll[time][pitch] >= 0.125:
                midi.addNote(0, 0, pitch + 21, time, 1, 100)  # channel=0, duration=1, volume=100

    with open(output_file, 'wb') as midi_file:
        midi.writeFile(midi_file)
        
piano_roll_to_midi(test_np.reshape((64,84,1)), tempo=120, output_file='test_org.mid')
piano_roll_to_midi(B_hat, tempo=120, output_file='test_gen.mid')



1/1 [==============================] - 0s 23ms/step


In [9]:
mf = midi.MidiFile()
mf.open('test_org.mid') 
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
print("Original Classic")
s.show('midi')

Original Classic 


In [10]:
mf = midi.MidiFile()
mf.open('test_gen.mid') 
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
print(" Classic to Jazz")
s.show('midi')

 Classic to Jazz
